In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,date
from pandas.tseries.holiday import USFederalHolidayCalendar
import os

In [30]:
base_path_DPMU = "..\\..\\data\\summer\\targets\\trimmed_targets"
all_months_DPMU = {} ##{mm:{a:, b:, c:}, mm:{a:,b:,c:}}

include_kmeans = 1
include_PCA = 0
include_holidays = 0
include_wknds = 0
include_dayofweek = 0
include_phase = 0
%store include_kmeans
%store include_PCA
%store include_holidays
%store include_wknds
%store include_dayofweek
%store include_phase

Stored 'include_kmeans' (int)
Stored 'include_PCA' (int)
Stored 'include_holidays' (int)
Stored 'include_wknds' (int)
Stored 'include_dayofweek' (int)
Stored 'include_phase' (int)


# Input (DPMU) Preprocessing (sends to X.csv)

In [3]:
#modifies raw (trimmed) csv to add time features and process the various phases/houses
def modify_csv(month_phase_csv, month_func, year_user):
    ##time modifications
    month_fill = 0
    if month_func == 'june':
        month_fill = 6
    elif month_func == 'july':
        month_fill = 7
    elif month_func == 'aug':
        month_fill = 8
    elif month_func == 'sept':
        month_fill = 9
    time = month_phase_csv['# timestamp']
    time = pd.Series(time)
    time = time.str.split(" ")
    for index in range(len(time)):
        del time[index][2]
    time
    month = []
    hour = []
    for index in range(len(time)):
        month.append(datetime.strptime(time[index][0],'%Y-%m-%d'))
        hour.append(datetime.strptime(time[index][1],'%H:%M:%S'))
    date = []
    timestamp = []

    for index in range(len(time)):
        for jindex in range(len(time[index])):
            if "-" in time[index][jindex]:
                date.append(time[index][jindex].split("-"))
            if ":" in time[index][jindex]:
                timestamp.append(time[index][jindex].split(":"))
    date = np.array(date)
    timestamp = np.array(timestamp)
    #separating the individual values into different list
    year = []
    month = []
    day = []
    for y in range(len(date)):
        year.append(date[y][0])
        month.append(date[y][1])
        day.append(date[y][2])

    #do special processing for the hours and minutes
    day_min = []
    for y in range(len(timestamp)):
        hour_min_added = int(timestamp[y][0]) * 60
        hour_min_added = hour_min_added + int(timestamp[y][1])
        day_min.append(hour_min_added) 

    ##code to overwrite 'January' outputs for csv's    
    year = [year_user] * len(date)
    month = [month_fill] * len(date)
        
    time_df = pd.DataFrame()
    time_df["year"] = year
    time_df["month"] = month
    time_df["day"] = day
    

    time_df["hour_min"] = day_min
        
    month_phase_mod = month_phase_csv.drop(['Meter5', '# timestamp'], axis=1)
    month_phase_mod = pd.concat([month_phase_mod,time_df], axis=1)
    #print (month_phase_mod)
    return month_phase_mod

In [4]:
#modifies the global all_months_DPMU. When this is over, that array will have all the necessary information to be concatenated into a giant array.
def sep_DPMU_features():
    #print (base_path_DPMU)
    for month in os.listdir(base_path_DPMU):
        month_path = os.path.join(base_path_DPMU,month)
        month_dict = {}
        for multi_voltage_output in os.listdir(month_path):
            full_csv_path = os.path.join(month_path,multi_voltage_output)
            #print(full_csv_path)
            cut_voltage = multi_voltage_output[:-4]
            #print(cut_voltage)
            letter = cut_voltage[-1:]
            #print(letter)
            dpmu_phase_month = "dpmu_phase" + letter + "_" + month
            raw_csv = pd.read_csv(full_csv_path)
            month_dict[letter] = modify_csv(raw_csv, month, 2018)
        #add month dictionary
        all_months_DPMU[month]=month_dict
    #return all the months it found in the base dir
    return list(all_months_DPMU.keys())
    

In [5]:
def DPMU(phase, dpmu_data, letter):
    phase_letter = ord(letter) - 65
    #print(phase_letter)
    for (columnName, columnData) in phase.iteritems():
        i=0
        if 'Meter' in columnName: ##meters
            split = columnName.split('Meter')
            house_name = f'Meter{letter}_'+ split[1]
            i+=1
            dpmu_values = pd.Series(columnData.values).str.replace("i", "j").apply(lambda x: abs(complex(x)))
            dpmu_data[house_name] = dpmu_values
        elif 'node' in columnName : ##nodes
            split_column_names = columnName.split('_')
            house_name = 'house'+str(split_column_names[phase_letter])
            dpmu_values = pd.Series(columnData.values).str.replace("i", "j").apply(lambda x: abs(complex(x)))
            dpmu_data[house_name] = dpmu_values
        else: ##timestamps, day_min
            if letter == 'A':
                dpmu_data[columnName] = pd.Series(columnData.values)
    return dpmu_data
    

In [6]:
def concat_all_DPMU():
    all_months_to_concat = []
    for month in all_months_DPMU:
        #print(month)
        dpmu_data_month = pd.DataFrame()
        for phase in all_months_DPMU[month]:
            #print (phase)
            dpmu_data_month = DPMU(all_months_DPMU[month][phase.upper()], dpmu_data_month, phase.upper())
        all_months_to_concat.append(dpmu_data_month)
    input_dataframe_DPMU = pd.DataFrame() ##concatenates all months, directly to input
    input_dataframe_DPMU = pd.concat(all_months_to_concat, ignore_index = True)
    return (input_dataframe_DPMU)
    
        

In [41]:
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
def sep_DPMU_clusters_kmeans(): #simoni
    # determined convergence to be 5 clusters .. see png
    #sum_of_squared_distances = []
    #K = range(1,37)
    #for k in K:
    #    print(k)
    #    k_means = KMeans(n_clusters=k)
    #    model = k_means.fit(in_csv)
    #    sum_of_squared_distances.append(k_means.inertia_)
    #    print(k_means.inertia_)
    #plt.plot(K, sum_of_squared_distances, 'bx-')
    #plt.xlabel('k')
    #plt.ylabel('sum_of_squared_distances')
    #plt.title('elbow method for optimal k')
    #plt.show()
    kmeans_df = in_csv
    kmeans_df_transpose = np.transpose(kmeans_df)
    k_means = KMeans(n_clusters=6)
    #model = k_means.fit(kmeans_df_transpose)
    model = k_means.fit_predict(kmeans_df_transpose)
    #cluster = k_means.fit_predict(kmeans_df_transpose)

#     dictionary = {}
#     for index in range(len(kmeans_df_transpose)): 
#         if cluster[index] in dictionary:
#             value = []
#             value = dictionary[cluster[index]]
#             value.append(cluster[index])
#             dictionary[cluster[index]] = value
#         else:
#             dictionary[cluster[index]]=cluster[index]
#     print(dictionary)
      
    print(model)
#     closest, _ = pairwise_distances_argmin_min(KMeans.cluster_centers_, X)
#     print(closest)
    #print(k_means.cluster_centers_)

#     labels = k_means.labels_
#     print(metrics.silhouette_score(kmeans_df, labels, metric = 'euclidean'))
#     print(metrics.calinski_harabasz_score(kmeans_df, labels))

In [20]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
def sep_DPMU_clusters_PCA(): #simoni
    pca_df = in_csv
    pca = PCA(n_components=37) # set to max components to determine convergence
    pca.fit(pca_df)
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('Number of components')
    plt.ylabel('Cumulative explained variance') # returns that best # of components for convergence is ~10

In [21]:
def phase_clustering(): #swetha
    print(in_csv)
    #only keep one house/meter from each phase
    in_csv.drop(in_csv.columns.difference(['MeterA_1', 'MeterB_1', 'MeterC_1', 'housenode1', 'housenode2', 'housenode3', 'year',
                                          'month', 'day', 'hour_min']), 1, inplace=True)
    #print phase_clustering_df

    

In [22]:
#return df modified to include flags for holidays
def add_holidays(): #swetha
    print(in_csv)
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start='2018-01-01', end='2018-12-31').to_pydatetime()

    holiday_array = []
    for i in range((in_csv.shape[0])):
        if datetime(int(in_csv.loc[in_csv.index[i], 'year']),
                   int(in_csv.loc[in_csv.index[i], 'month']),
                   int(in_csv.loc[in_csv.index[i], 'day'])) in holidays:
            holiday_array.append(1)
        else:
            holiday_array.append(0)
            
    in_csv["is_holiday"] = holiday_array


In [23]:
#return df modified to include flags for weekend
def add_weekends(): #swetha

    wknd_array = []
    for i in range((in_csv.shape[0])):
        wkday = date(int(in_csv.loc[in_csv.index[i], 'year']),
                   int(in_csv.loc[in_csv.index[i], 'month']),
                   int(in_csv.loc[in_csv.index[i], 'day'])).weekday()
        is_wknd = 1 if wkday>=4 else 0
        wknd_array.append(is_wknd)
    in_csv["is_weekend"] = wknd_array


In [24]:
#return df modified to include flags for weekend
def add_dayofweek(): #swetha

    wk_array = []
    for i in range((in_csv.shape[0])):
        wkday = date(int(in_csv.loc[in_csv.index[i], 'year']),
                   int(in_csv.loc[in_csv.index[i], 'month']),
                   int(in_csv.loc[in_csv.index[i], 'day'])).isoweekday()
        wk_array.append(wkday)

    in_csv["day_of_week"] = wk_array


In [25]:
##all_months_DPMU will have a dictionary of csv's for each month, each phase
sep_DPMU_features()

['aug', 'july', 'june', 'sept']

In [26]:
##input_dataframe_DPMU will have large dataframe of input data
send_input_to_csv = pd.DataFrame()
send_input_to_csv = concat_all_DPMU()

In [27]:
send_input_to_csv

,MeterA_2,MeterA_1,MeterA_3,MeterA_4,housenode1,housenode4,housenode7,housenode13,housenode10,housenode22,...,MeterC_3,MeterC_4,house3,house6,house9,house15,house12,house24,house21,house18
0,2376.409889,2378.388636,2361.400016,2357.538734,2363.994761,2376.991693,2375.718498,2352.767045,2361.400016,2361.418434,...,2347.430794,2343.286737,2366.926658,2366.504003,2366.229047,2337.377650,2347.430794,2344.827519,2347.430794,2342.868207
1,2380.907281,2382.797347,2371.481343,2369.147260,2369.150113,2380.564999,2378.824397,2364.411194,2371.481343,2371.381328,...,2350.315321,2346.584561,2368.194516,2368.069600,2367.731763,2340.824483,2350.315321,2347.744618,2350.315321,2346.217019
2,2377.256861,2379.415364,2366.150157,2362.635628,2365.298299,2377.165092,2375.486746,2355.295777,2366.150157,2365.965524,...,2347.366674,2343.390828,2368.214272,2367.751656,2367.378270,2337.614698,2347.366674,2344.726159,2347.366674,2343.008353
3,2374.778306,2376.835701,2360.276611,2355.968593,2363.462683,2374.634968,2372.936101,2347.306906,2360.276611,2360.103638,...,2346.915101,2342.973333,2368.147718,2367.509425,2367.188528,2337.128129,2346.915101,2344.249188,2346.915101,2342.826055
4,2374.294670,2376.486046,2360.364542,2356.340245,2361.685581,2374.155190,2372.415367,2348.156111,2360.364542,2360.259043,...,2350.219551,2346.321728,2370.321920,2369.072138,2368.743645,2340.504174,2350.219551,2347.680284,2350.219551,2346.162014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175675,2383.647127,2383.949621,2364.816449,2361.909454,2381.794340,2384.738434,2384.456407,2357.466568,2364.816449,2365.072928,...,2387.879512,2387.002581,2389.002002,2388.000478,2388.188000,2385.968149,2387.879512,2387.750663,2387.879512,2386.800404
175676,2384.341416,2384.643960,2366.016259,2363.111796,2382.212336,2385.452623,2385.210038,2358.636611,2366.016259,2366.251903,...,2381.532573,2380.660222,2385.728215,2384.553452,2384.738626,2379.636839,2381.532573,2381.440251,2381.523886,2380.458055
175677,2388.071887,2388.354823,2373.152195,2370.222813,2385.795089,2389.302478,2389.148854,2365.685649,2373.152195,2373.062116,...,2370.071818,2369.202664,2379.323677,2378.633987,2378.813187,2368.182870,2370.071818,2369.865101,2370.063124,2369.014147
175678,2388.068581,2388.391098,2373.609684,2370.815485,2385.984918,2389.277919,2389.113869,2366.339327,2373.609684,2373.440121,...,2366.610109,2365.695425,2378.270558,2377.354002,2377.516891,2364.671980,2366.610109,2366.403122,2366.610109,2365.488313


In [28]:
list(send_input_to_csv.columns)

['MeterA_2',
 'MeterA_1',
 'MeterA_3',
 'MeterA_4',
 'housenode1',
 'housenode4',
 'housenode7',
 'housenode13',
 'housenode10',
 'housenode22',
 'housenode19',
 'housenode16',
 'year',
 'month',
 'day',
 'hour_min',
 'MeterB_2',
 'MeterB_1',
 'MeterB_3',
 'MeterB_4',
 'house2',
 'house5',
 'house8',
 'house14',
 'house11',
 'house23',
 'house20',
 'house17',
 'MeterC_2',
 'MeterC_1',
 'MeterC_3',
 'MeterC_4',
 'house3',
 'house6',
 'house9',
 'house15',
 'house12',
 'house24',
 'house21',
 'house18']

In [29]:
send_input_to_csv = send_input_to_csv[['MeterA_1', 'MeterA_2', 'MeterA_3', 'MeterA_4', 
                                       'MeterB_1', 'MeterB_2', 'MeterB_3', 'MeterB_4',
                                      'MeterC_1', 'MeterC_2', 'MeterC_3', 'MeterC_4',
                                      'housenode1','housenode4','housenode7',
                                       'housenode13','housenode10','housenode22',
                                       'housenode19','housenode16',
                                      'house2','house5', 'house8','house14',
                                       'house11','house23','house20','house17',
                                       'house3','house6','house9','house15','house12',
                                       'house24','house21','house18',
                                       'year','month','day','hour_min']]

# Combining Phases

In [35]:
send_input_to_csv2 = pd.DataFrame()
Meter_1 = [send_input_to_csv['MeterA_1'],send_input_to_csv['MeterB_1'],send_input_to_csv['MeterC_1']]
send_input_to_csv2['Meter_1'] = pd.concat(Meter_1)
Meter_2 = [send_input_to_csv['MeterA_2'] + send_input_to_csv['MeterB_2'] + send_input_to_csv['MeterC_2']]
send_input_to_csv2['Meter_2'] = pd.concat(Meter_2)
Meter_3 = [send_input_to_csv['MeterA_3'] + send_input_to_csv['MeterB_3'] + send_input_to_csv['MeterC_3']]
send_input_to_csv2['Meter_3'] = pd.concat(Meter_3)
Meter_4 = [send_input_to_csv['MeterA_4'] + send_input_to_csv['MeterB_4'] + send_input_to_csv['MeterC_4']]
send_input_to_csv2['Meter_4'] = pd.concat(Meter_4)
node_1 = [send_input_to_csv['housenode1'] + send_input_to_csv['house2'] + send_input_to_csv['house3']]
send_input_to_csv2['node_1'] = pd.concat(node_1)
node_2 = [send_input_to_csv['housenode4'] + send_input_to_csv['house5'] + send_input_to_csv['house6']]
send_input_to_csv2['node_2'] = pd.concat(node_2)
node_3 = [send_input_to_csv['housenode7'] + send_input_to_csv['house8'] + send_input_to_csv['house9']]
send_input_to_csv2['node_3'] = pd.concat(node_3)
node_4 = [send_input_to_csv['housenode10'] + send_input_to_csv['house11'] + send_input_to_csv['house12']]
send_input_to_csv2['node_4'] = pd.concat(node_4)
node_5 = [send_input_to_csv['housenode13'] + send_input_to_csv['house14'] + send_input_to_csv['house15']]
send_input_to_csv2['node_5'] = pd.concat(node_5)
node_6 = [send_input_to_csv['housenode16'] + send_input_to_csv['house17'] + send_input_to_csv['house18']]
send_input_to_csv2['node_6'] = pd.concat(node_6)
node_7 = [send_input_to_csv['housenode19'] + send_input_to_csv['house20'] + send_input_to_csv['house21']]
send_input_to_csv2['node_7'] = pd.concat(node_7)
node_8 = [send_input_to_csv['housenode22'] + send_input_to_csv['house23'] + send_input_to_csv['house24']]
send_input_to_csv2['node_8'] = pd.concat(node_8)

send_input_to_csv2

,Meter_1,Meter_2,Meter_3,Meter_4,node_1,node_2,node_3,node_4,node_5,node_6,node_7,node_8
0,2378.388636,7155.932462,7127.739313,7120.019002,7137.874792,7148.479991,7147.032322,7127.739313,7108.816288,7117.477425,7127.739251,7124.987557
1,2382.797347,7161.782529,7139.807053,7133.678037,7142.804311,7154.108860,7152.282466,7139.807053,7122.654862,7132.421682,7139.807002,7137.530892
2,2379.415364,7156.911062,7135.632566,7128.726007,7128.230528,7150.591978,7148.794810,7135.632566,7115.029710,7127.807326,7135.632507,7133.464549
3,2376.835701,7154.189169,7131.028521,7123.653212,7126.085164,7147.308212,7145.432870,7131.028521,7108.555104,7123.020742,7131.028460,7128.868706
4,2376.486046,7154.056799,7132.222172,7125.083724,7123.567303,7147.343134,7145.451681,7132.222172,7110.490846,7124.449044,7132.222111,7130.160615
...,...,...,...,...,...,...,...,...,...,...,...,...
175675,2392.486273,7164.974492,7142.731132,7139.330930,7148.181833,7160.094766,7158.299544,7142.731132,7133.243209,7138.623361,7142.731112,7141.123072
175676,2389.079528,7165.973173,7144.944684,7141.571197,7149.475569,7160.993206,7159.174666,7144.944684,7135.470746,7140.842922,7144.935971,7143.397980
175677,2383.157287,7168.592803,7150.303796,7146.929306,7151.999184,7163.639934,7161.856821,7150.303796,7140.787849,7146.204300,7150.295071,7149.574398
175678,2381.661285,7171.191061,7153.714697,7150.433792,7156.975331,7166.228495,7164.499307,7153.714697,7144.356996,7149.742018,7153.714657,7153.104108


# No clustering

In [50]:
in_csv = pd.DataFrame()
in_csv = send_input_to_csv2.copy()

#12 inputs Original
#in_csv = in_csv.drop(['house23','house24','house17','house18','house14','house15','house20', 'house21','house11','house12','house2','house3', 'house5', 'house6','house8','house9','MeterA_2','MeterA_3','MeterA_4','MeterB_1','MeterB_3','MeterA_4','MeterC_1','MeterC_2','MeterC_4','year','month','day','hour_min'], axis=1)

#12 inputs different nodes
#in_csv = in_csv.drop(['housenode22','house24','housenode16','house18','housenode13','house15','housenode19', 'house21','housenode10','house12','housenode1','house3', 'housenode4', 'house6','housenode7','house9','MeterA_2','MeterA_3','MeterA_4','MeterB_1','MeterB_3','MeterA_4','MeterC_1','MeterC_2','MeterC_4','year','month','day','hour_min'], axis=1)

#6 inputs
#in_csv = in_csv.drop(['housenode22','house23','house24','housenode16','house18','housenode13','house14','house15','housenode19', 'house21','housenode10','house12','housenode1','house2','house3', 'housenode4', 'house6','housenode7','house8','house9','MeterA_1','MeterA_2','MeterA_3','MeterA_4','MeterB_1','MeterB_3','MeterA_4','MeterC_1','MeterC_2','MeterC_3','MeterC_4','year','month','day','hour_min'], axis=1)

# foundation inverse
#in_csv = in_csv.drop(['year','month','day','hour_min'], axis=1)

#3 inputs
#in_csv = in_csv.drop(['housenode22','house23','house24','housenode16','house17','house18','housenode13','house14','house15','housenode19','house20', 'house21','housenode10','house12','housenode1','house2','house3', 'housenode4', 'house6','housenode7','house8','house9','MeterA_1','MeterA_2','MeterA_3','MeterA_4','MeterB_1','MeterB_2','MeterB_3','MeterA_4','MeterC_1','MeterC_2','MeterC_3','MeterC_4','year','month','day','hour_min'], axis=1)


send_input_to_csv.to_csv('../../data/input/X.csv')

In [51]:
in_csv

,Meter_1,Meter_2,Meter_3,Meter_4,node_1,node_2,node_3,node_4,node_5,node_6,node_7,node_8
0,2378.388636,7155.932462,7127.739313,7120.019002,7137.874792,7148.479991,7147.032322,7127.739313,7108.816288,7117.477425,7127.739251,7124.987557
1,2382.797347,7161.782529,7139.807053,7133.678037,7142.804311,7154.108860,7152.282466,7139.807053,7122.654862,7132.421682,7139.807002,7137.530892
2,2379.415364,7156.911062,7135.632566,7128.726007,7128.230528,7150.591978,7148.794810,7135.632566,7115.029710,7127.807326,7135.632507,7133.464549
3,2376.835701,7154.189169,7131.028521,7123.653212,7126.085164,7147.308212,7145.432870,7131.028521,7108.555104,7123.020742,7131.028460,7128.868706
4,2376.486046,7154.056799,7132.222172,7125.083724,7123.567303,7147.343134,7145.451681,7132.222172,7110.490846,7124.449044,7132.222111,7130.160615
...,...,...,...,...,...,...,...,...,...,...,...,...
175675,2392.486273,7164.974492,7142.731132,7139.330930,7148.181833,7160.094766,7158.299544,7142.731132,7133.243209,7138.623361,7142.731112,7141.123072
175676,2389.079528,7165.973173,7144.944684,7141.571197,7149.475569,7160.993206,7159.174666,7144.944684,7135.470746,7140.842922,7144.935971,7143.397980
175677,2383.157287,7168.592803,7150.303796,7146.929306,7151.999184,7163.639934,7161.856821,7150.303796,7140.787849,7146.204300,7150.295071,7149.574398
175678,2381.661285,7171.191061,7153.714697,7150.433792,7156.975331,7166.228495,7164.499307,7153.714697,7144.356996,7149.742018,7153.714657,7153.104108


# K-means, PCA, phase, holidays, weekends

In [52]:
csv_name = "X"


if (include_kmeans):
    sep_DPMU_clusters_kmeans()
    csv_name = csv_name + "_kmeans"

if(include_PCA):
    sep_DPMU_clusters_PCA()
    csv_name = csv_name + "_pca"

if(include_phase):
    phase_clustering()
    csv_name = csv_name + "_phase"
    
if(include_holidays):
    add_holidays()
    csv_name = csv_name + "_holidays"
    
if(include_wknds):
    add_weekends()
    csv_name = csv_name + "_weekends"
    
if(include_dayofweek):
    add_dayofweek()
    csv_name = csv_name + "_weekdays"


[1 5 0 4 3 2 2 0 4 4 0 0]


In [53]:
csv_name

'X_kmeans'

In [54]:
in_csv.columns

Index(['Meter_1', 'Meter_2', 'Meter_3', 'Meter_4', 'node_1', 'node_2',
       'node_3', 'node_4', 'node_5', 'node_6', 'node_7', 'node_8'],
      dtype='object')

In [55]:
#first
# in_csv = in_csv.drop(['MeterA_1','MeterA_4','MeterB_2','MeterC_2', 'housenode4', 'housenode7', 'housenode13', 'housenode10',
#         'housenode22', 'housenode19', 'housenode16','house5','house8', 'house14', 'house11', 'house23', 'house20', 'house17','house9', 'house15', 'house12', 'house24',
#        'house21', 'house18'], axis=1)

#second
# in_csv = in_csv.drop(['MeterA_2','MeterA_3','MeterB_1','MeterC_2', 'housenode4', 'housenode7', 'housenode13', 'housenode10',
#        'housenode22', 'housenode19', 'housenode16','house5','house8', 'house14', 'house11', 'house23', 'house20', 'house17','house9', 'house15', 'house12', 'house24',
#        'house21', 'house18'], axis=1)

#6 clusters on combined phases
in_csv = in_csv.drop(['node_2','node_3','node_4','node_6', 'node_7', 'node_8'], axis=1)


In [56]:
in_csv

,Meter_1,Meter_2,Meter_3,Meter_4,node_1,node_5
0,2378.388636,7155.932462,7127.739313,7120.019002,7137.874792,7108.816288
1,2382.797347,7161.782529,7139.807053,7133.678037,7142.804311,7122.654862
2,2379.415364,7156.911062,7135.632566,7128.726007,7128.230528,7115.029710
3,2376.835701,7154.189169,7131.028521,7123.653212,7126.085164,7108.555104
4,2376.486046,7154.056799,7132.222172,7125.083724,7123.567303,7110.490846
...,...,...,...,...,...,...
175675,2392.486273,7164.974492,7142.731132,7139.330930,7148.181833,7133.243209
175676,2389.079528,7165.973173,7144.944684,7141.571197,7149.475569,7135.470746
175677,2383.157287,7168.592803,7150.303796,7146.929306,7151.999184,7140.787849
175678,2381.661285,7171.191061,7153.714697,7150.433792,7156.975331,7144.356996


In [57]:
csv_name = csv_name + ".csv"
in_csv.to_csv('../../data/input/' + csv_name)

# Output (Grid) Preprocessing (sends to Y.csv)

In [60]:
base_path_Player = "..\\..\\data\\summer\\input_csv"

In [61]:
def set_output():
    month_players = []
    for month in os.listdir(base_path_Player):
        month_path = os.path.join(base_path_Player,month)
        input_player_df = pd.DataFrame()
        for filename in os.listdir(month_path):
            if str(filename).startswith('25.csv'):
                continue
            player = pd.read_csv(os.path.join(month_path, filename))
            player['grid'] = player['grid'].str.replace("i", "j").apply(lambda x: complex(x))
            node_id = player['id'].loc[0]
            column_name_p = 'p_'+str(node_id)
            column_name_q = 'q_'+str(node_id)
        #change .head() value to target dataframe row number
            input_player_df[column_name_p] = player['grid'].apply(lambda x: x.real).head(44640)
            input_player_df[column_name_q] = player['grid'].apply(lambda x: x.imag).head(44640)
        month_players.append(input_player_df)
    output_dataframe_Player = pd.DataFrame() ##concatenates all months, directly to input
    output_dataframe_Player = pd.concat(month_players, ignore_index = True)
    return (output_dataframe_Player)
    
    

In [62]:
##input_dataframe_DPMU will have large dataframe of input data
send_targets_to_csv = pd.DataFrame()
send_targets_to_csv = set_output()

In [63]:
send_targets_to_csv

,p_1,q_1,p_10,q_10,p_11,q_11,p_12,q_12,p_13,q_13,...,p_5,q_5,p_6,q_6,p_7,q_7,p_8,q_8,p_9,q_9
0,43770.0,18273.479083,49815.0,19677.968025,6210.0,2993.529277,7680.0,2977.205699,61020.0,25629.453761,...,53790.0,22680.824631,67515.0,26725.875676,29070.0,12766.083147,6045.0,2538.376656,5310.0,1768.574303
1,43800.0,16056.165241,49830.0,21169.506826,6210.0,2503.047231,7410.0,3187.775742,60975.0,24463.259976,...,53640.0,18591.367838,67395.0,24229.587465,39240.0,16830.434105,6090.0,2080.689498,5295.0,1960.017239
2,43740.0,16725.190261,49980.0,18571.507872,6210.0,2406.866873,7410.0,3368.241776,95295.0,34245.168859,...,53670.0,17841.890713,67035.0,23688.227789,39525.0,14630.069312,6165.0,2053.104821,5265.0,2332.545638
3,43695.0,14533.893706,49920.0,21483.136368,6210.0,2552.831448,7410.0,2727.305834,111255.0,45610.401766,...,53565.0,25357.082993,67005.0,25112.807910,39675.0,15125.471091,8325.0,3691.961411,5280.0,1817.387203
4,43635.0,18926.971373,49875.0,19129.801554,6210.0,2285.612698,8385.0,2785.115072,105225.0,42562.705246,...,53520.0,24179.304713,67050.0,27466.505811,39645.0,16417.639968,7950.0,2996.814233,5265.0,2177.816589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175675,6465.0,2822.317617,45765.0,18524.166195,50505.0,22528.098381,8430.0,2789.272489,57450.0,25219.495588,...,4650.0,1807.317228,64635.0,27118.997672,4740.0,1824.514606,41190.0,13817.778604,6705.0,2637.722707
175676,7725.0,3085.881665,45735.0,20028.965065,24030.0,10193.354166,23640.0,10681.441361,57855.0,25297.044348,...,4635.0,1981.916417,62775.0,29347.144545,4725.0,1975.152094,39915.0,16853.690903,6645.0,2488.146655
175677,7650.0,3636.386829,45810.0,17385.362109,18000.0,6288.798710,38730.0,16854.617167,59265.0,23744.529706,...,4635.0,2003.650515,62595.0,29119.896786,3105.0,1308.767508,39375.0,18577.764183,6645.0,2787.186256
175678,7635.0,2981.949705,46065.0,18604.384766,8415.0,2868.802547,39315.0,18257.810943,58860.0,21173.453924,...,4620.0,1985.184652,61470.0,25781.111189,2745.0,1154.465979,39315.0,16184.443484,6645.0,3169.747109


In [66]:
send_targets_to_csv2 = pd.DataFrame()
node_1_p = [send_targets_to_csv['p_1'],send_targets_to_csv['p_2'],send_targets_to_csv['p_3']]
send_targets_to_csv2['node_1_p'] = pd.concat(node_1_p)
node_1_q = [send_targets_to_csv['q_1'],send_targets_to_csv['q_2'],send_targets_to_csv['q_3']]
send_targets_to_csv2['node_1_q'] = pd.concat(node_1_q)
node_2_p = [send_targets_to_csv['p_4'],send_targets_to_csv['p_5'],send_targets_to_csv['p_6']]
send_targets_to_csv2['node_2_p'] = pd.concat(node_2_p)
node_2_q = [send_targets_to_csv['q_4'],send_targets_to_csv['q_5'],send_targets_to_csv['q_6']]
send_targets_to_csv2['node_2_q'] = pd.concat(node_2_q)
node_3_p = [send_targets_to_csv['p_7'],send_targets_to_csv['p_8'],send_targets_to_csv['p_9']]
send_targets_to_csv2['node_3_p'] = pd.concat(node_3_p)
node_3_q = [send_targets_to_csv['q_7'],send_targets_to_csv['q_8'],send_targets_to_csv['q_9']]
send_targets_to_csv2['node_3_q'] = pd.concat(node_3_q)
node_4_p = [send_targets_to_csv['p_10'],send_targets_to_csv['p_11'],send_targets_to_csv['p_12']]
send_targets_to_csv2['node_4_p'] = pd.concat(node_4_p)
node_4_q = [send_targets_to_csv['q_10'],send_targets_to_csv['q_11'],send_targets_to_csv['q_12']]
send_targets_to_csv2['node_4_q'] = pd.concat(node_4_q)
node_5_p = [send_targets_to_csv['p_13'],send_targets_to_csv['p_14'],send_targets_to_csv['p_15']]
send_targets_to_csv2['node_5_p'] = pd.concat(node_5_p)
node_5_q = [send_targets_to_csv['q_13'],send_targets_to_csv['q_14'],send_targets_to_csv['q_15']]
send_targets_to_csv2['node_5_q'] = pd.concat(node_5_q)
node_6_p = [send_targets_to_csv['p_16'],send_targets_to_csv['p_17'],send_targets_to_csv['p_18']]
send_targets_to_csv2['node_6_p'] = pd.concat(node_6_p)
node_6_q = [send_targets_to_csv['q_16'],send_targets_to_csv['q_17'],send_targets_to_csv['q_18']]
send_targets_to_csv2['node_6_q'] = pd.concat(node_6_q)
node_7_p = [send_targets_to_csv['p_19'],send_targets_to_csv['p_20'],send_targets_to_csv['p_21']]
send_targets_to_csv2['node_7_p'] = pd.concat(node_7_p)
node_7_q = [send_targets_to_csv['q_19'],send_targets_to_csv['q_20'],send_targets_to_csv['q_21']]
send_targets_to_csv2['node_7_q'] = pd.concat(node_7_q)
node_8_p = [send_targets_to_csv['p_22'],send_targets_to_csv['p_23'],send_targets_to_csv['p_24']]
send_targets_to_csv2['node_8_p'] = pd.concat(node_8_p)
node_8_q = [send_targets_to_csv['q_22'],send_targets_to_csv['q_23'],send_targets_to_csv['q_24']]
send_targets_to_csv2['node_8_q'] = pd.concat(node_8_q)

send_targets_to_csv2

,node_1_p,node_1_q,node_2_p,node_2_q,node_3_p,node_3_q,node_4_p,node_4_q,node_5_p,node_5_q,node_6_p,node_6_q,node_7_p,node_7_q,node_8_p,node_8_q
0,43770.0,18273.479083,9660.0,3490.733137,29070.0,12766.083147,49815.0,19677.968025,61020.0,25629.453761,46140.0,17696.078875,62685.0,30263.821815,6675.0,2488.392321
1,43800.0,16056.165241,9660.0,3445.014877,39240.0,16830.434105,49830.0,21169.506826,60975.0,24463.259976,12780.0,5269.704706,62745.0,21761.031687,6690.0,2868.202125
2,43740.0,16725.190261,10005.0,4722.576966,39525.0,14630.069312,49980.0,18571.507872,95295.0,34245.168859,8445.0,2891.942199,62850.0,20977.394143,6705.0,2741.756852
3,43695.0,14533.893706,11325.0,4085.282270,39675.0,15125.471091,49920.0,21483.136368,111255.0,45610.401766,4575.0,1950.501319,62820.0,21822.210564,6705.0,2263.392609
4,43635.0,18926.971373,11820.0,4711.582334,39645.0,16417.639968,49875.0,19129.801554,105225.0,42562.705246,4590.0,1926.180148,62565.0,26201.760424,6705.0,2400.397675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175675,13005.0,5074.891741,64635.0,27118.997672,6705.0,2637.722707,8430.0,2789.272489,14055.0,5499.088703,5640.0,2110.142649,35160.0,14955.582133,7575.0,3528.450339
175676,12840.0,4524.903966,62775.0,29347.144545,6645.0,2488.146655,23640.0,10681.441361,14100.0,4911.450062,5535.0,2396.111113,35100.0,11903.985461,7500.0,2884.670000
175677,12630.0,5917.374173,62595.0,29119.896786,6645.0,2787.186256,38730.0,16854.617167,13995.0,4654.670362,5625.0,1983.306483,35085.0,15226.698392,7575.0,3224.922283
175678,11325.0,5153.927900,61470.0,25781.111189,6645.0,3169.747109,39315.0,18257.810943,13935.0,4924.326497,5580.0,2105.863156,35085.0,15996.274233,7590.0,3563.160127


In [67]:
send_targets_to_csv2.to_csv('../../data/outputs/Y.csv')